<a href="https://colab.research.google.com/github/thisissamuca/GOES_16/blob/main/GET_MASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usado para obter a máscara de nuvem reclassificada a partir do produto ABI-L2-ACMF

In [ ]:
pip install xarray rasterio affine satpy netCDF4 affine cartopy

In [ ]:
# Montando drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Máscara de nuvem reclassificada

In [ ]:
from affine import Affine
import os
import netCDF4
import cv2
import cartopy.crs
import numpy as np
import xarray as xr
import rasterio
from satpy import Scene
from typing import List, Dict, Optional, Generator
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Configuração para suprimir avisos não essenciais
warnings.filterwarnings('ignore')

class NetCDFBatchProcessor:
    """Classe para processamento em lote de arquivos NetCDF com dados de satélite."""

    def __init__(self, base_path: str = '/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS'): # Pasta de arquivos que armazena os produtos ACMF
        self.base_path = base_path

    def list_product_files(self, product: str) -> List[str]:
        """Lista todos os arquivos disponíveis para um produto específico."""
        product_path = os.path.join(self.base_path, product, 'netCDF')
        return sorted([f for f in os.listdir(product_path) if f.endswith('.nc')])

    def get_variables(self, product: str, filename: str) -> Dict[int, str]:
        """Retorna um dicionário com as variáveis disponíveis em um arquivo NetCDF."""
        file_path = os.path.join(self.base_path, product, 'netCDF', filename)

        with netCDF4.Dataset(file_path) as dataset:
            variables = {i: var for i, var in enumerate(dataset.variables.keys())}

        return variables

    def load_dataset(self, product: str, filename: str) -> xr.Dataset:
        """Carrega um arquivo NetCDF como xarray Dataset."""
        file_path = os.path.join(self.base_path, product, 'netCDF', filename)
        return xr.open_dataset(file_path)

class ImageBatchProcessor:
    """Classe para processamento em lote de imagens derivadas de dados de satélite."""

    @staticmethod
    def reclassify_array(data: np.ndarray, mapping: Dict[int, int]) -> np.ndarray:
        """Reclassifica um array numpy baseado em um dicionário de mapeamento (versão otimizada)."""
        # Versão otimizada usando np.where para melhor desempenho
        result = np.zeros_like(data, dtype=np.float32)
        for key, value in mapping.items():
            result = np.where(data == key, value, result)
        return result

    @staticmethod
    def safe_convert_to_float(data: np.ndarray) -> np.ndarray:
        """Converte valores do array para float de forma segura (versão vetorizada)."""
        # Versão otimizada usando vectorize com cache
        float_func = np.vectorize(lambda x: float(x) if isinstance(x, (int, float)) else np.nan,
                                otypes=[np.float32])
        return float_func(data)

    @staticmethod
    def resize_image(image: np.ndarray, target_shape: tuple = (5424, 5424)) -> np.ndarray:
        """Redimensiona imagem mantendo proporções (usando interpolação bilinear)."""
        return cv2.resize(image, target_shape, interpolation=cv2.INTER_NEAREST)

class GeoTIFFBatchExporter:
    """Classe para exportação em lote de dados para formato GeoTIFF."""

    @staticmethod
    def create_transform(area) -> Affine:
        """Cria transformação afim a partir da área de dados."""
        res_x = area.pixel_size_x
        res_y = -area.pixel_size_y
        xmin, ymax = area.area_extent[0], area.area_extent[3]
        return Affine.translation(xmin, ymax) * Affine.scale(res_x, res_y)

    @staticmethod
    def export_to_geotiff(data: np.ndarray, crs, transform, output_path: str):
        """Exporta array numpy para arquivo GeoTIFF."""
        profile = {
            'driver': 'GTiff',
            'height': data.shape[0],
            'width': data.shape[1],
            'count': 1,
            'dtype': data.dtype,
            'crs': crs,
            'transform': transform,
            'compress': 'lzw',  # Adicionado compressão para reduzir tamanho dos arquivos
            'tiled': True      # Melhora performance para arquivos grandes
        }

        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data, 1)

def process_single_file(ncdf_processor, img_processor, exporter, product: str, filename: str,
                       reclass_map: Dict, target_shape: tuple, output_dir: str):
    """Processa um único arquivo NetCDF e exporta para GeoTIFF."""
    try:
        start_time = time.time()

        # Carrega e processa os dados
        ds = ncdf_processor.load_dataset(product, filename)
        first_var = list(ds.variables.keys())[0]
        variable_data = ds[first_var].data

        # Processamento da imagem
        reclassified = img_processor.reclassify_array(variable_data, reclass_map)
        float_array = img_processor.safe_convert_to_float(reclassified)
        resized_image = img_processor.resize_image(float_array, target_shape)

        # Configuração do sistema de coordenadas
        file_path = os.path.join(ncdf_processor.base_path, product, 'netCDF', filename)
        scn = Scene(filenames=[file_path], reader='abi_l2_nc')
        scn.load(['BCM'])

        area = scn['BCM'].attrs['area']
        transform = exporter.create_transform(area)
        crs = area.to_cartopy_crs()

        # Gera nome do arquivo de saída
        output_filename = f"{os.path.splitext(filename)[0]}_RECLASSIFIED.tif"
        output_path = os.path.join(output_dir, output_filename)

        # Exportação
        exporter.export_to_geotiff(resized_image, crs, transform, output_path)

        processing_time = time.time() - start_time
        return (filename, True, processing_time)

    except Exception as e:
        return (filename, False, str(e))

def batch_process_files(product: str, output_dir: str, max_workers: int = 4):
    """Processa todos os arquivos de um produto em paralelo."""
    # Configurações
    RECLASS_MAP = {1: 0, 0: 1}
    TARGET_SHAPE = (5424, 5424)

    # Inicialização dos processadores
    ncdf_processor = NetCDFBatchProcessor()
    img_processor = ImageBatchProcessor()
    exporter = GeoTIFFBatchExporter()

    # Obtém lista de arquivos
    files = ncdf_processor.list_product_files(product)
    if not files:
        raise FileNotFoundError(f"Nenhum arquivo encontrado para o produto {product}")

    print(f"Iniciando processamento de {len(files)} arquivos...")

    # Cria diretório de saída se não existir
    os.makedirs(output_dir, exist_ok=True)

    # Processamento em paralelo
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for filename in files:
            future = executor.submit(
                process_single_file,
                ncdf_processor,
                img_processor,
                exporter,
                product,
                filename,
                RECLASS_MAP,
                TARGET_SHAPE,
                output_dir
            )
            futures.append(future)

        # Monitora progresso e resultados
        success_count = 0
        for future in as_completed(futures):
            filename, status, result = future.result()
            if status is True:
                print(f"Arquivo {filename} processado com sucesso em {result:.2f} segundos")
                success_count += 1
            else:
                print(f"Erro ao processar {filename}: {result}")

    print(f"\nProcessamento concluído! {success_count}/{len(files)} arquivos processados com sucesso.")

if __name__ == "__main__":
    # Configurações para execução
    PRODUCT = "ABI-L2-ACMF"
    OUTPUT_DIR = "/content/drive/MyDrive/NBR/ARQUIVOS_PROCESSADOS/MASCARA_RECLASSIFICADA"
    MAX_WORKERS = 1  # Ajuste conforme os recursos disponíveis

    # Executa o processamento em lote
    start_total = time.time()
    batch_process_files(PRODUCT, OUTPUT_DIR, MAX_WORKERS)
    total_time = time.time() - start_total

    print(f"\nTempo total de processamento: {total_time:.2f} segundos")

Iniciando processamento de 12 arquivos...
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201521300164_e20201521309472_c20201521310161.nc processado com sucesso em 14.93 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201521310164_e20201521319472_c20201521320225.nc processado com sucesso em 11.53 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201521320164_e20201521329472_c20201521330186.nc processado com sucesso em 11.59 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201521330164_e20201521339472_c20201521340226.nc processado com sucesso em 11.36 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201521340164_e20201521349471_c20201521350201.nc processado com sucesso em 10.15 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201521350164_e20201521359471_c20201521400228.nc processado com sucesso em 11.51 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201531300167_e20201531309475_c20201531310180.nc processado com sucesso em 11.48 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201531310167_e20201531319475_c20201531320232.nc processado com sucesso 